<a href="https://colab.research.google.com/github/CalculatedContent/WeightWatcher/blob/master/examples/WW-RandMats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# WW Rand Mats

How the weightwatcher PL estimator performs on random matrices






In [ ]:
# Suppress the powerlaw package warnings
# "powerlaw.py:700: RuntimeWarning: divide by zero encountered in true_divide"
# "powerlaw.py:700: RuntimeWarning: invalid value encountered in true_divide"
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

### Import WeightWatcher


In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install weightwatcher

In [ ]:
import logging

import weightwatcher as ww
import torchvision.models as models

logger = logging.getLogger(ww.__name__)
logger.setLevel(logging.INFO)

ww.__version__

###  Create a Dummy Model

In [ ]:
import weightwatcher as ww
import torchvision.models as model

model = models.vgg11(pretrained=True)
watcher = ww.WeightWatcher(model=model)
details = watcher.describe()

In [ ]:
details['Q']=details.N/details.M

In [ ]:
details

### Q ~= 1, 2, 4, 6, 21

In [ ]:
import powerlaw
Qs , alphas = [], []
for layer_id in [28, 13, 31, 25, 2]:
    Q = details[details.layer_id == layer_id].Q.to_numpy()[0]
    
    esd = watcher.get_ESD(layer=layer_id, random=True)
    fit = powerlaw.Fit(esd, xmax=np.max(esd))
    alpha = fit.alpha
    print("Q={}  alpha={:0.2}".format(Q, alpha))
    Qs.append(Q)
    alphas.append(alpha)